In [1]:
using FFTW

In [45]:
?FFTW.dct

```
dct(A [, dims])
```

Performs a multidimensional type-II discrete cosine transform (DCT) of the array `A`, using the unitary normalization of the DCT. The optional `dims` argument specifies an iterable subset of dimensions (e.g. an integer, range, tuple, or array) to transform along.  Most efficient if the size of `A` along the transformed dimensions is a product of small primes; see [`nextprod`](@ref). See also [`plan_dct`](@ref) for even greater efficiency.


In [46]:
# Equivalent to python scipy.fftpack.dct([1,2,3], norm='ortho')
dct([1,2,3])

3-element Vector{Float64}:
  3.4641016151377544
 -1.414213562373095
  9.064933036736789e-17

In [48]:
dct([[1,3,4,4,5] [1,2,3,4,5]])

5×2 Matrix{Float64}:
 10.1193      0.632456
 -4.1912      0.262866
 -0.58541    -0.58541
 -0.826948   -0.425325
  0.0854102   0.0854102

In [49]:
[[1,3,4,4,5] [1,2,3,4,5]]

5×2 Matrix{Int64}:
 1  1
 3  2
 4  3
 4  4
 5  5

In [ ]:
dct([1,2,3])